<a href="https://colab.research.google.com/github/ameerfayiz/ML-notebooks/blob/main/Zero_shot_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.8/577.8 KB 28.2 MB/s eta 0:00:00


In [23]:
import json
from transformers import pipeline
import time
import torch

torch.cuda.is_available()

labels = ['travel', 'friends','series','movies','entertainment','technology','family' ,'algorithms','cooking', 'job' , 'dancing','health','happiness','activity','social']

# classifier = pipeline ("zero-shot-classification", model = "facebook/bart-large-mnli")                  # 1.63 Gb 0.4384075531592736 per label
# classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")      # 558  Mb 0.313440519112807 per label
# classifier = pipeline("zero-shot-classification", model="BaptisteDoyen/camembert-base-xnli")            # 443 Mb 0.12476613338177021 per label
# classifier = pipeline("zero-shot-classification", model="typeform/distilbert-base-uncased-mnli")        # 268 Mb 0.05824783215155968 per label
# classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-1")                # 890 Mb 0.20728054413428673 per label 
# classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli") # 369 Mb 0.3029702296623817 per label 
# classifier = pipeline("zero-shot-classification", model="cross-encoder/nli-deberta-v3-base")            # 738 Mb 0.30590915496532733 per label
# classifier = pipeline("zero-shot-classification", model="cross-encoder/nli-deberta-v3-small")           # 568 Mb 0.15964309252225434 per label
# classifier = pipeline("zero-shot-classification", model="cross-encoder/nli-MiniLM2-L6-H768")            # 329 Mb 0.055275693306556115 per label
classifier = pipeline("zero-shot-classification", model="typeform/mobilebert-uncased-mnli")             # 99 Mb 0.03554045275637978 per label
# classifier = pipeline("zero-shot-classification", model="Recognai/zeroshot_selectra_medium")            # 163 Mb 0.03785876310788668 per label
# classifier = pipeline("zero-shot-classification", model="Recognai/zeroshot_selectra_small")             # 90 Mb 0.019344867978777207 per label



print("model loaded!")


def classify(prompt,multi_label=True,loop=10):
    begin = time.time()
    for i in range(loop):
      result = classifier(prompt, labels,multi_label=multi_label)
    end = time.time()
    print(json.dumps(result,indent="  "))
    # total time taken
    gap = (end - begin)/10
    print(f"Total runtime of the program is {gap} is, {gap/len(labels)} per label")


classify("To avoid plagiarism issues, please refer to the code on the Google Colab")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


model loaded!
{
  "sequence": "To avoid plagiarism issues, please refer to the code on the Google Colab",
  "labels": [
    "technology",
    "social",
    "activity",
    "algorithms",
    "travel",
    "series",
    "health",
    "family",
    "job",
    "cooking",
    "movies",
    "entertainment",
    "friends",
    "dancing",
    "happiness"
  ],
  "scores": [
    0.5289000272750854,
    0.25394248962402344,
    0.22862575948238373,
    0.21224497258663177,
    0.10091692954301834,
    0.09477368742227554,
    0.06666842103004456,
    0.06201644986867905,
    0.041554179042577744,
    0.031228413805365562,
    0.02283315360546112,
    0.017696712166070938,
    0.013975279405713081,
    0.00899865385144949,
    0.0058839586563408375
  ]
}
Total runtime of the program is 0.6104580163955688 is, 0.04069720109303792 per label


In [24]:
classify("Got up early at 04:40 to drive my mom to the airport Went back to sleep til 09:00",loop=1)

{
  "sequence": "Got up early at 04:40 to drive my mom to the airport Went back to sleep til 09:00",
  "labels": [
    "social",
    "activity",
    "family",
    "entertainment",
    "health",
    "travel",
    "algorithms",
    "job",
    "happiness",
    "series",
    "cooking",
    "movies",
    "technology",
    "friends",
    "dancing"
  ],
  "scores": [
    0.912135660648346,
    0.9059642553329468,
    0.6507245898246765,
    0.6373061537742615,
    0.5867769122123718,
    0.5659762024879456,
    0.19942346215248108,
    0.16412216424942017,
    0.1610601544380188,
    0.1586294025182724,
    0.09640653431415558,
    0.07738259434700012,
    0.04231642559170723,
    0.02215701900422573,
    0.021515414118766785
  ]
}
Total runtime of the program is 0.0603635311126709 is, 0.004024235407511393 per label


In [28]:
classify("Watched Netflix series Kaleidiscope with jake",loop=1)

{
  "sequence": "Watched Netflix series Kaleidiscope with jake",
  "labels": [
    "series",
    "activity",
    "travel",
    "social",
    "entertainment",
    "family",
    "algorithms",
    "movies",
    "job",
    "friends",
    "technology",
    "dancing",
    "happiness",
    "health",
    "cooking"
  ],
  "scores": [
    0.8246340751647949,
    0.8004382848739624,
    0.5025167465209961,
    0.5000870823860168,
    0.4723190367221832,
    0.2099054753780365,
    0.1343669444322586,
    0.07104060798883438,
    0.05750013515353203,
    0.04688529670238495,
    0.034444063901901245,
    0.03394891694188118,
    0.02608572505414486,
    0.020000776275992393,
    0.016639888286590576
  ]
}
Total runtime of the program is 0.05171303749084473 is, 0.003447535832722982 per label
